In [ ]:
from pathlib import Path

import uproot
import awkward as ak
from coffea import nanoevents

import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh.processors.utils import GEN_FLAGS, PDGID, pad_val

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = Path("../../")

plot_dir = MAIN_DIR / "plots/TriggerStudy/24Nov6"
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
events = nanoevents.NanoEventsFactory.from_root(
    "root://cmseos.fnal.gov//store/user/lpcdihiggsboost/NanoAOD_v12_ParT/rkansal/2022/HHbbtt/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV/241028_235514/0000/Run3Summer22NanoAODv12_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

In [ ]:
np.min(ak.flatten(ak.firsts(events.FatJet.pt), axis=None))

In [ ]:
plt.hist(
    ak.flatten(ak.firsts(events.FatJet.pt), axis=None), np.linspace(0, 1000, 101), histtype="step"
)

In [ ]:
events.HLT.fields

In [ ]:
events.FatJet.fields

In [ ]:
genparts = events.GenPart[events.GenPart.hasFlags(GEN_FLAGS)]
higgs = genparts[genparts.pdgId == PDGID.H]
higgs_children = higgs.children
bb = higgs_children[np.abs(higgs_children.pdgId) == PDGID.b]
taus = higgs_children[np.abs(higgs_children.pdgId) == PDGID.tau]
tau_children = ak.flatten(taus.children, axis=2)

In [ ]:
ak.any(ak.is_none(higgs_children.pdgId[:, :, 0]))

In [ ]:
ak.flatten(bb.pt, axis=2).to_numpy().data

In [ ]:
pad_val(ak.flatten(bb.pt, axis=2), 2, axis=1)

In [ ]:
pad_val(higgs_children.pdgId[:, :, 0], 2, axis=1)

In [ ]:
# iterate through the children in photon scattering events to get final daughter particles
for i in range(5):
    tau_mask = ak.any(np.abs(tau_children.pdgId) == TAU_PDGID, axis=2)
    if not ak.any(tau_mask):
        print(f"Done after {i} iterations")
        break

    tau_children_children = ak.flatten(
        tau_children[np.abs(tau_children.pdgId) == TAU_PDGID].children, axis=3
    )

    # use a where condition to get next layer of children for photon scattering events
    tau_children = ak.where(tau_mask, tau_children_children, tau_children)

In [ ]:
def _sum_taus(taut):
    return ak.sum(taut, axis=1)


# check neutral and charged pion IDs for hadronic taus
tauh = _sum_taus(
    ak.any([ak.any(np.abs(tau_children.pdgId) == pid, axis=2) for pid in PI_PDGIDS], axis=0)
)
taumu = _sum_taus(ak.any(np.abs(tau_children.pdgId) == MU_PDGID, axis=2))
taue = _sum_taus(ak.any(np.abs(tau_children.pdgId) == ELE_PDGID, axis=2))

tauhh = tauh == 2
tauhmu = (tauh == 1) & (taumu == 1)
tauhe = (tauh == 1) & (taue == 1)

In [ ]:
mhh = (higgs[:, 0] + higgs[:, 1]).mass.to_numpy()
weights = events.genWeight.to_numpy()

In [ ]:
# Tau triggers Run 3 https://twiki.cern.ch/twiki/bin/view/CMSPublic/Run3TauHLT

triggers_dict = {
    "tauhh": {
        "mask": tauhh,
        "label": r"$\tau_h\tau_h$",
        "triggers": {
            "ParticleNetBB": events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35,
            "Di-tau": events.HLT.DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1
            | events.HLT.DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60
            | events.HLT.DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75,
            "ParticleNetBB | PFJet425": (
                events.HLT.AK8PFJet425_SoftDropMass40
                | events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
            ),
            "ParticleNetBB | PFJet425 | Di-tau": (
                events.HLT.AK8PFJet425_SoftDropMass40
                | events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1
                | events.HLT.DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60
                | events.HLT.DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75
            ),
        },
    },
    "tauhmu": {
        "mask": tauhmu,
        "label": r"$\tau_h\mu$",
        "triggers": {
            "ParticleNetBB": events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35,
            "ParticleNetBB | IsoMu24 | Mu50": (
                events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.IsoMu24
                | events.HLT.Mu50
            ),
            "ParticleNetBB | IsoMu24 | Mu50 | Mu-tau": (
                events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.IsoMu24
                | events.HLT.Mu50
                | events.HLT.IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1
                | events.HLT.IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_CrossL1
                | events.HLT.IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_TightID_CrossL1
            ),
            "ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet": (
                events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.IsoMu24
                | events.HLT.Mu50
                | events.HLT.IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1
                | events.HLT.IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1
                | events.HLT.IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_CrossL1
                | events.HLT.IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_TightID_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60_CrossL1
                | events.HLT.IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75_CrossL1
            ),
        },
    },
    "tauhe": {
        "mask": tauhe,
        "label": r"$\tau_he$",
        "triggers": {
            "ParticleNetBB": events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35,
            "ParticleNetBB | EGamma": (
                events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.Ele30_WPTight_Gsf
                | events.HLT.Ele115_CaloIdVT_GsfTrkIdT
                | events.HLT.Ele50_CaloIdVT_GsfTrkIdT_PFJet165
                | events.HLT.Photon200
            ),
            "ParticleNetBB | EGamma | E-tau": (
                events.HLT.AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35
                | events.HLT.Ele30_WPTight_Gsf
                | events.HLT.Ele115_CaloIdVT_GsfTrkIdT
                | events.HLT.Ele50_CaloIdVT_GsfTrkIdT_PFJet165
                | events.HLT.Photon200
                | events.HLT.Ele24_eta2p1_WPTight_Gsf_LooseDeepTauPFTauHPS30_eta2p1_CrossL1
                | events.HLT.Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTauHPS30_eta2p1_CrossL1
            ),
        },
    },
}

In [ ]:
plt.rcParams.update({"font.size": 14})

for cat, vals in triggers_dict.items():
    (mask, label, triggers) = vals.values()

    fig, (ax, rax) = plt.subplots(
        2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
    )

    bins = np.linspace(250, 1000, 50)
    hists = {"All": np.histogram(mhh[mask], bins=bins, weights=weights[mask])}
    ratios = {}

    hep.histplot(
        hists["All"],
        yerr=False,
        label="All",
        ax=ax,
    )

    colours = plt.cm.tab10.colors[1:]

    for key, c in zip(triggers.keys(), colours):
        print(key)
        hists[key] = np.histogram(
            mhh[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
        )
        ratios[key] = hists[key][0] / hists["All"][0]

        hep.histplot(
            hists[key],
            yerr=False,
            label=key,
            ax=ax,
            color=c,
        )

        hep.histplot(
            (ratios[key], bins),
            yerr=False,
            label=key,
            ax=rax,
            histtype="errorbar",
            color=c,
            capsize=4,
        )

    ax.set_ylabel("Events [A.U.]")
    ax.legend()
    ax.set_title(label)

    rax.legend()
    rax.grid(axis="y")
    rax.set_xlabel(r"$m_{HH}$ [GeV]")
    rax.set_ylabel("Triggered / All")

    hep.cms.label(ax=ax, data=False, year=2022, com="13.6")

    plt.savefig(plot_dir / f"mhh_{cat}.pdf", bbox_inches="tight")
    plt.show()